## Description

This notebook downloads and crops the data used in the final dataset. To use, simply run all the cells

In [4]:
import os
import sys
import shutil
import PIL
import requests
import PIL.Image as Image
from tqdm import tqdm
import numpy as np
import pandas as pd

def load_image(filename):
    return Image.open(filename).convert('RGB') #this conversion is crucial as PIL will open png files with either 3 or 4 channels depending on encoding

def center_crop(img):
    '''Algorithm derived from nvidia preprocessor'''
    crop = np.min(img.shape[:2])
    img = img[(img.shape[0]-crop)//2:(img.shape[0]+crop)//2,(img.shape[1]-crop)//2:(img.shape[1]+crop)//2]
    img = Image.fromarray(img,'RGB')
    return img

def resize_and_crop(fp,width,height):
    img=np.asarray(load_image(fp),dtype=np.uint8)
    img=center_crop(img)
    
#     if img.size[1]<height or img.size[0]<width:
#         print('here')
#         print(fp)
    
    img=img.resize((width,height),Image.LANCZOS)
    img=np.array(img)
    
    canvas = np.zeros([width,width,3],dtype=np.uint8)
    canvas[(width-height)//2: (width+height)//2,:]=img
    return Image.fromarray(canvas )

        #img.save(f'filen.jpg')


def download_images(reddit_csv):
    print('Downloading and Cropping Images')
    fp ='Data'
    fpt = os.path.join('Data','temp')
    if not os.path.exists(fp):
        os.mkdir(fp)
    os.mkdir(fpt)
    fpw= os.path.join(fp,'Weapon')
    fpp =os.path.join(fp,'People')
    os.mkdir(fpw)
    os.mkdir(fpp)
    df=pd.read_csv(reddit_csv)
    
    post_fix = {'Success':0,'Failed':0}

    
    pbar = tqdm(df.iterrows(),total=len(df))
    
    for index, row in pbar:
        image_url = row.url
        filename = os.path.join(fpt,row['url'].split('/')[-1])
        label = row.label
        filename=os.path.join(fpt,row['url'].split('/')[-1])
        
        try:
            r= requests.get(image_url,stream=True)
        except:
            post_fix['Failed']+=1 
            continue
        if r.status_code==200:
            r.raw.decode_content = True
            with open(filename,'wb') as f:
                shutil.copyfileobj(r.raw,f)
            post_fix['Success']+=1
        else:
            post_fix['Failed']+=1
        
        ##Crop
        canvas=resize_and_crop(filename,256,256)
        if label =='People':
            fn2 =os.path.join(fpp,row['url'].split('/')[-1])
        if label =='Weapon':
             fn2 =os.path.join(fpw,row['url'].split('/')[-1])
        canvas.save(fn2)
        os.remove(filename)
        pbar.set_postfix(post_fix)
    pbar.close()
    os.rmdir(fpt)
    

In [6]:
download_images('data_download.csv')

  0%|                                                  | 0/4595 [00:00<?, ?it/s]/var/folders/zx/_x1xg_4n2hj8d_6dx7nmrm5m0000gn/T/ipykernel_32885/1664118427.py:29: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img=img.resize((width,height),Image.LANCZOS)
  0%|                     | 9/4595 [00:03<25:33,  2.99it/s, Success=9, Failed=0]

KeyboardInterrupt

